# 21강 워드 임베딩 (2) Word2Vec 실습 1회차: 임베딩 모델 훈련

2025-12-03

## 0. 환경 설정

In [ ]:
# 필요한 라이브러리 설치
# %pip install datasets scikit-learn pandas numpy matplotlib seaborn konlpy
# %pip install gensim

In [ ]:
# 라이브러리 임포트
import pandas as pd
from datasets import load_dataset, load_from_disk
from collections import Counter
from tqdm import tqdm
from gensim.models import Word2Vec, FastText
from konlpy.tag import Okt

print("라이브러리 로드 완료!")

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


라이브러리 로드 완료!


## 1. 코퍼스 준비

### 1.1 NSMC 데이터셋 로드

In [4]:
# NSMC 데이터셋 로드
print("NSMC 데이터셋 로드 중...")
dataset = load_dataset("Blpeng/nsmc")

NSMC 데이터셋 로드 중...


Repo card metadata block was not found. Setting CardData to empty.


### 1.2. 텍스트 전처리 및 토큰화

In [ ]:
data_path = '../data/nsmc-okt'

In [ ]:
# 형태소 분석기
tagger = Okt()

def preprocess(row):
    document = row['document']
    if document is not None:
        output = tagger.morphs(document)
    else:
        output = []
    
    return {'morphs': output}

dataset = dataset.map(preprocess)
dataset = dataset.remove_columns(['Unnamed: 0'])

# 파일로 쓰기
dataset.save_to_disk(data_path)

In [ ]:
# 기존 파일에서 읽어오기
# dataset = load_from_disk(data_path)

### 1.3. Train/Test 분리

In [ ]:
train_corpus = dataset['train']['morphs']
test_corpus = dataset['test']['morphs']

## 2. 임베딩 모델 훈련

### 2.1. 준비: 단어 빈도 계산 (Optional)

In [ ]:
freq_path = '../data/nsmc-okt-word-freq.csv'

In [ ]:
word_freq = Counter()
for morphs in tqdm(train_corpus):
    word_freq += Counter(morphs)


# 파일로 쓰기
pd.DataFrame(word_freq.most_common(), columns=('word', 'freq')).to_csv(freq_path, index=None)

In [ ]:
# 기존 파일에서 읽어오기
# word_freq = Counter(pd.read_csv(freq_path).set_index('word')['freq'].to_dict())

In [9]:
word_freq.most_common(10)

[('.', 67784),
 ('이', 56108),
 ('영화', 50737),
 ('의', 30963),
 ('..', 29058),
 ('가', 27571),
 ('에', 26772),
 ('을', 23237),
 ('...', 22801),
 ('도', 21426)]

### 2.2. 모델 초기화, 훈련, 저장

In [13]:
!mkdir -p ../models

seed = 1046 # BWV 1046
workers = 1 # for reproducibility
epochs = 5 # default: 5

models = {
    'Word2Vec': {
        'CBOW': Word2Vec(sg=0, seed=seed, workers=workers),
        'SG': Word2Vec(sg=1, seed=seed, workers=workers),
    },
    'FastText': {
        'CBOW': FastText(sg=0, seed=seed, workers=workers),
        'SG': FastText(sg=1, seed=seed, workers=workers),
    }
}

for class_ in models:
    for algorithm in models[class_]:
        print(f'{class_} {algorithm} 모델 훈련 시작...')
        model = models[class_][algorithm]
        model.build_vocab_from_freq(word_freq, keep_raw_vocab=True)
        model.train(corpus_iterable=train_corpus, total_examples=len(train_corpus), epochs=epochs)
        models[class_][algorithm] = model
        model_path = f'../models/nsmc_{class_.lower()}_{algorithm.lower()}.model'
        model.save(model_path)
        print('단어 벡터 저장 완료!', model_path)
        print('='*20)

Word2Vec CBOW 모델 훈련 시작...
단어 벡터 저장 완료! ../models/nsmc_word2vec_cbow.model
Word2Vec SG 모델 훈련 시작...
단어 벡터 저장 완료! ../models/nsmc_word2vec_sg.model
FastText CBOW 모델 훈련 시작...
단어 벡터 저장 완료! ../models/nsmc_fasttext_cbow.model
FastText SG 모델 훈련 시작...
단어 벡터 저장 완료! ../models/nsmc_fasttext_sg.model


**주의**: 저장된 임베딩 모델 파일은 매우 크기가 크므로 `.gitignore`에 추가해서 동기화를 막을 것!